# Języki i Biblioteki Analizy Danych
## Laboratorium 6.: Generatory
#### mgr inż. Zbigniew Kaleta

* Generatorów używamy, aby oszczędzić pamięć (a także czas potrzebny na jej alokację).
* Zysk wydajności powstaje przez ominięcie potrzeby tworzenia tymczasowych struktur pośrednich w pamięci, gdy zamiast tego możemy przeiterować kolejno po elementach i finalnie zapisać tylko te, które są potrzebne.
* Generator to "funkcja po której można iterować", w każdej iteracji zwracająca kolejną wartość

generator zwraca wiele wartości. Zwraca pojedyńczo wartości. Zysk pamięci. Tworzymy za pomoocą:


### polecenie ``yield``

Funkcja zwracająca listę kolejnych elementów ciągu Fibonacciego nie przekraczających zadanej wartości.

In [1]:
def fib(n=100):
    result = []
    a, b = 0, 1
    while b < n:
        result += [b]
        a, b = b, a + b
    return result

In [2]:
fib()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

To samo w postaci generatora

In [3]:
def fib(n=100):
    a, b = 0, 1
    while b < n:
        yield b  # <- please note 'yield' in place of appending to list
        a, b = b, a + b

In [4]:
fib()

<generator object fib at 0x0000028256A36810>

In [5]:
# typowy sposób użycia generatora
for i in fib():
    print(i)

1
1
2
3
5
8
13
21
34
55
89


In [6]:
# a jeśli chcę mieć wszystkie wartości w liście...
list(fib())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

In [1]:
def f():
    yield 2
    yield 3
    return 4  # return powoduje rzucenie StopIteration z wiadomością 4; ten generator nie wygeneruje ani 4, ani 5
    yield 5

for i in f():
    print(i)

2
3


yield umożliwia komunikację dwukierunkową, nie tylko zwraca a może też otrzymać odpowiedź 

In [8]:
def xrange(n):
    i = 0
    while i < n:
        yield i
        i += 1
        
list(xrange(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
def xrange(n):
    i = 0
    while i < n:
        j = yield i  # yield pozwala na komunikację dwukierunkową - "zwracam wartość i czekam na odpowiedź"
        if j is not None:
            i = j
        else:
            i += 1
        
g = xrange(10)
print(next(g))

0


przkazanie, wysłanie liczby do generatora

In [3]:
print(next(g))

1


In [4]:
print(g.send(1))  # wysyłam odpowiedź do yielda i przekazuję sterowanie do generatora (tak jak w przypadku wywołania next)

1


Patrz: coroutines... ale potoki (pipeline) generatorów są lepsze

### Krótka historia range'a

skąd się wzieła xrange() nazwa 

In [12]:
%%python2

print(range(10))
print(xrange(10))

Couldn't find program: 'python2'


In [15]:
%%python3

# print(range(10))
print(xrange(10))

range(0, 10)


In [20]:
print(list(xrange(10))) # z [] nie dziala tylko jest obiekt generator w środku

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Generator expression

In [21]:
g = (i*i for i in range(10))  # takie same możliwości jak list comprehension, tylko generator

In [22]:
g

<generator object <genexpr> at 0x0000028257406EA0>

In [23]:
type(g)

generator

In [24]:
for i in g:
    print(i)
    if i>4:
        break

0
1
4
9


In [25]:
for i in g:
    print(i)

16
25
36
49
64
81


wydajność generatorów 

In [26]:
from time import perf_counter

In [27]:
start_time = perf_counter()
for i in [a**2 for a in range(20_000_000)]:
    break
print("DONE")
print("Time elapsed: {:.5f}s".format(perf_counter() - start_time))

DONE
Time elapsed: 18.43421s


In [28]:
start_time = perf_counter()
for i in (a**2 for a in range(20_000_000)):
    break
print("DONE")
print("Time elapsed: {:.5f}s".format(perf_counter() - start_time))

DONE
Time elapsed: 0.00117s


Jedna z ważniejszych przewag generatora nad listą to możliwość przerwania generowania w dowolnym momencie. Nie marnujemy czasu na obliczenie wartości, które nie będą potrzebne.

Generator może również zawierać w sobie pętlę nieskończoną, a "obowiązek" jej przerwania będzie spoczywał na użytkowniku tego generatora.

Wynik testu z pełną iteracją po tej kolekcji (dla 2 mln elementów):

Lista: 0.50881s

Generator: 0.49612s

In [29]:
g = (i*i for i in range(10) if i%2 for j in range(2))
list(g)

[1, 1, 9, 9, 25, 25, 49, 49, 81, 81]

### Lista czy generator?

Jeżeli elementów do zwrócenia jest mało, to się nie zastanawiamy, tylko piszemy co nam wygodniej.

In [30]:
def f():
    return [1, 2, 3]
    
def g():
    yield 1
    yield 2
    yield 3

goenerator nie obsługuje indeksowania 

Jeśli interesuje nas tylko jedna iteracja po elementach, to właściwie na jedno wychodzi.

In [31]:
l = f()
for i in l:
    print(i)

1
2
3


In [32]:
l = g()
for i in l:
    print(i)

1
2
3


Jeżeli potrzebujemy iterować wielokrotnie (albo tym bardziej potrzebujemy dostępu swobodnego), to lista jest nieodzowna...

In [33]:
l = f()
for i in l:
    print(i)
for i in l:
    print(i)

1
2
3
1
2
3


In [34]:
l = g()
for i in l:
    print(i)
for i in l:
    print(i)

1
2
3


... przy czym może ją sobie stworzyć kod wywołujący.

In [35]:
l = list(g())
for i in l:
    print(i)
for i in l:
    print(i)

1
2
3
1
2
3


Dwukrotne wywołanie generatora będzie niewydajne, ale może się przydać, jeżeli pracujemy na sprzęcie z bardzo ograniczoną pamięcią, a nasz ciąg jest bardzo długi.

In [36]:
l = g()
for i in l:
    print(i)
l = g()
for i in l:
    print(i)

1
2
3
1
2
3


### Funkcja czy generator?

In [37]:
def f():
    return 1

print(f())

1


In [38]:
def f():
    yield 1

print(f())

<generator object f at 0x000002825755D620>


In [39]:
print(next(f()))  # niewygodne w użyciu; generator nie zastąpi zwykłej funkcji

1


In [40]:
def f():
    yield from [1, 2, 3]
    
print(f)
print(f())

<function f at 0x00000282576A8040>
<generator object f at 0x000002825755F3E0>


### Chaining (pipelining) generatorów

Potok generatorów polega na tym, że jeden generator pobiera wartości z drugiego i zwraca je dalej, gdzie może czekać kolejny generator. Na początku potoku może być generator, ale równie dobrze może to być cokolwiek innego iterowalnego, jak lista, czy plik.

Poszczególne składowe potoku mogą zmniejszać liczbę elementów (np. odfiltrowywać liczby nieparzyste), zostawiać taką samą (np. podnosić liczby do kwadratu) lub zwiększać (np. rozbijać liczbę na poszczególne cyfry).

In [41]:
def gen1():
    for i in range(100000):
        yield i
    
def filter1(gen):
    for i in gen:
        if i%2:
            yield i

def filter2(gen):
    for i in gen:
        if not i%11:
            yield i

for i in filter2(filter1(gen1())):
    print(i)

11
33
55
77
99
121
143
165
187
209
231
253
275
297
319
341
363
385
407
429
451
473
495
517
539
561
583
605
627
649
671
693
715
737
759
781
803
825
847
869
891
913
935
957
979
1001
1023
1045
1067
1089
1111
1133
1155
1177
1199
1221
1243
1265
1287
1309
1331
1353
1375
1397
1419
1441
1463
1485
1507
1529
1551
1573
1595
1617
1639
1661
1683
1705
1727
1749
1771
1793
1815
1837
1859
1881
1903
1925
1947
1969
1991
2013
2035
2057
2079
2101
2123
2145
2167
2189
2211
2233
2255
2277
2299
2321
2343
2365
2387
2409
2431
2453
2475
2497
2519
2541
2563
2585
2607
2629
2651
2673
2695
2717
2739
2761
2783
2805
2827
2849
2871
2893
2915
2937
2959
2981
3003
3025
3047
3069
3091
3113
3135
3157
3179
3201
3223
3245
3267
3289
3311
3333
3355
3377
3399
3421
3443
3465
3487
3509
3531
3553
3575
3597
3619
3641
3663
3685
3707
3729
3751
3773
3795
3817
3839
3861
3883
3905
3927
3949
3971
3993
4015
4037
4059
4081
4103
4125
4147
4169
4191
4213
4235
4257
4279
4301
4323
4345
4367
4389
4411
4433
4455
4477
4499
4521
4543
4565
4587
4609


Gdyby każda z przedstawionych wyżej funkcji zwracała listę, to nadal by wszystko działało, tylko w każdym momencie potrzebowalibyśmy mieć dwie listy (lista zwrócona przez poprzedni element potoku i aktualnie tworzona). Przy dużych rozmiarach tych list zysk pamięciowy, a może również wydajnościowy, będzie zauważalny.